* `crossmatch_TOI_with_GaiaDR3.ipynb` attempts to cross-match based on position and magnitude difference.
* here result of previous cross-match from exofop or vizier is used

In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [2]:
import sys
sys.path.insert(0, '../code')

# TOI catalog

In [23]:
from catalog import get_tois

tois = get_tois()
# tois.columns.tolist()

In [24]:
tois[tois.Comments.apply(lambda x: 'young' in str(x))]

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg
732,146520535,942.01,NaN,3,5,5,3,4,4,4,...,NaN,0.814000,0.097250,5,2019-07-24,2020-05-20,2022-12-14 12:09:24,Probable young K dwarf. Transit is believable ...,76.649625,-20.245614
2902,330135981,3604.01,NaN,3,4,4,3,2,4,4,...,NaN,NaN,NaN,"16,17,56,57",2021-06-23,2023-02-01,2023-04-17 12:02:58,V-shaped; found in faint-star QLP search; no s...,333.501042,56.918981
4902,308300731,5743.01,NaN,3,4,3,3,4,4,4,...,NaN,0.898000,0.109025,"26,53,79,80",2022-09-01,2024-08-25,2024-08-26 00:00:00,radius possibly underestimated; potentially yo...,267.904333,24.427456
5099,236785891,5956.01,TIC 236785891.01,3,4,3,3,2,4,4,...,NaN,0.483975,0.021842,"56,57",2022-11-16,2023-02-03,2023-08-10 00:00:00,V-shaped; large; young host star; ~5 sigma odd...,283.317250,59.817658
5389,435878153,6258.01,NaN,2,4,4,2,4,4,4,...,NaN,0.368918,0.020209,"17,57",2023-04-06,2023-04-06,2024-02-23 00:00:00,low SNR; asymmetric transit shape; flaring pos...,13.085375,20.938875
5806,253864764,6678.01,NaN,1,4,1,1,1,4,4,...,NaN,0.770000,0.092773,67,2023-09-21,2023-09-21,2023-09-25 12:14:10,rotational modulation at 7-8 days may indicate...,283.536292,-36.656858


In [5]:
tois.shape

(6160, 64)

# cross-match/query TOI info from exofop

* CAUTION: might hit rate limit set by exofop

In [4]:
from catalog import get_tfop_info, get_params_from_tfop

tfop_info = get_tfop_info("TOI-4364")
tfop_info['basic_info']

{'tic_id': '4070275',
 'star_names': 'TIC 4070275, 2MASS J05201599-0414220, APASS 757852, Gaia DR2 3210444215030339584, TOI-4364, UCAC4 429-009115, WISE J052016.03-041421.5',
 'confirmed_planets': '',
 'k2_campaign': '',
 'tic_contamination_ratio': '0.165363'}

In [16]:
def get_gaiaid(tfop_info):
    names = tfop_info['basic_info'].get('star_names').split(', ')
    try:
        gaia = [i for i in names if i[:4]=='Gaia']
        return int(gaia[0].split()[-1])
    except Exception as e:
        print(e)

In [15]:
get_gaiaid(tfop_info)

3210444215030339584

# cross-match with vizier

* possibly limit is not as strict as in exofop server

In [49]:
cols = ['ra_deg','dec_deg']
toi_coords = tois[cols]

In [17]:
try:
    import swifter
except:
    !{sys.executable} -m pip install swifter==1.3.4
    import swifter
assert swifter.__version__ == "1.3.4"

In [59]:
import warnings
warnings.simplefilter("ignore", category="UnitsWarning")
from catalog import Target

class TargetWithProcessing(Target):
    def process_target(self, catalog='GaiaDR3'):
        # Perform the Vizier query and process results
        res = self.query_vizier_param(catalog)
        if len(set(res.values())) == 1:
            return set(res.values()).pop()
        return None  
 
def get_GaiaDR3_id_from_vizier(df_coords, param='Gaia'):
    return df_coords.apply(lambda coord: TargetWithProcessing(*coord, verbose=False).process_target(), axis=1)

In [ ]:
tois['GaiaDR3_vizier'] = get_GaiaDR3_id_from_vizier(toi_coords)

# cross-match with exofop

In [ ]:
def get_GaiaDR3_id_from_exofop(df_coords, param='Gaia', key='I/355/gaiadr3'):
    """try key='I/345/gaia2'
    """
    return df_coords.swifter.apply(lambda coord: Target(*coord, verbose=False)\
            .query_vizier_param(param)\
            .get(key), axis=1)

In [ ]:
tois['GaiaDR3_exofop'] = get_GaiaDR3_id_from_exofop(toi_coords)

In [11]:
tois.to_csv('../data/TOIs2.csv', index=False)

/tmp/ipykernel_3827076/3963823051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
